# FRB example

In [16]:
# imports
from importlib import reload
import os
from pkg_resources import resource_filename

import numpy as np

import pandas

from astropy.coordinates import SkyCoord
#from astropy.io import fits

from astropath import bayesian
from astropath import localization

# Dummy FRB (based on 180924)

## Coord

In [2]:
frb_coord = SkyCoord('21h44m25.255s -40d54m00.10s', frame='icrs')
frb_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (326.10522917, -40.90002778)>

## Error ellipse

In [34]:
#eellipse = dict(a=0.3, b=0.1, theta=45.)
eellipse = dict(a=0.1, b=0.1, theta=0.)

## Localization dict

### See astropath/localization.py for the Data model

In [35]:
localiz = dict(type='eellipse', frb_coord=frb_coord, frb_eellipse=eellipse)

In [28]:
assert localization.vette_localization(localiz)

# Candidates

## Load

In [7]:
cand_file = os.path.join(resource_filename('astropath', 'data'), 'frb_example', 'frb180924_candidates.csv')

In [8]:
candidates = pandas.read_csv(cand_file, index_col=0)
candidates.head()

,ra,dec,half_light,VLT_FORS2_g
3,326.101738,-40.899771,0.458869,25.295416
7,326.105365,-40.900239,1.308629,21.319569
8,326.104186,-40.900180,0.814683,24.272838
9,326.106237,-40.899279,0.501247,25.474828


## Coords

In [9]:
c_coords = SkyCoord(ra=candidates.ra, dec=candidates.dec, unit='deg')
c_coords

<SkyCoord (ICRS): (ra, dec) in deg
    [(326.1017384 , -40.89977098), (326.10536507, -40.90023897),
     (326.10418611, -40.90018023), (326.10623701, -40.8992786 )]>

# Priors

## Guiding dict

In [10]:
offset_prior = dict(method='exp', 
                    max=6.,
                   ang_size=candidates.half_light.values)
priors = dict(offset=offset_prior, 
              O='inverse', 
              U=0., 
              name='Adopted')

## Calculate raw

In [11]:
# Raw priors
raw_prior_Oi = bayesian.raw_prior_Oi(priors['O'], 
                                     candidates.VLT_FORS2_g.values, 
                                     half_light=candidates.half_light.values)

In [12]:
candidates['P_O_raw'] = raw_prior_Oi
candidates

,ra,dec,half_light,VLT_FORS2_g,P_O_raw
3,326.101738,-40.899771,0.458869,25.295416,73.150261
7,326.105365,-40.900239,1.308629,21.319569,2017.091915
8,326.104186,-40.900180,0.814683,24.272838,157.983590
9,326.106237,-40.899279,0.501247,25.474828,64.277492


## Normalize

In [13]:
candidates['P_O'] = bayesian.renorm_priors(candidates.P_O_raw.values, priors['U'])

In [14]:
candidates

,ra,dec,half_light,VLT_FORS2_g,P_O_raw,P_O
3,326.101738,-40.899771,0.458869,25.295416,73.150261,0.031633
7,326.105365,-40.900239,1.308629,21.319569,2017.091915,0.872255
8,326.104186,-40.900180,0.814683,24.272838,157.983590,0.068317
9,326.106237,-40.899279,0.501247,25.474828,64.277492,0.027796


# Calculate $p(x|O)$

## Original

In [36]:
reload(bayesian)
p_xOi = bayesian.px_Oi(30.,  # box radius for grid, in arcsec
                       frb_coord,
                       eellipse,
                       c_coords,
                       priors['offset'])
p_xOi

array([0.00000000e+00, 5.15677111e-02, 6.97344054e-03, 9.29664168e-20])

In [31]:
reload(localization)
reload(bayesian)
#
p_xOi = bayesian.px_Oi_fixed(30.,  # box radius for grid, in arcsec
                       localiz,
                       c_coords,
                       priors['offset'])
p_xOi

array([0.00000000e+00, 6.25655634e-02, 3.24391542e-03, 1.81710365e-08])

In [21]:
candidates['p_xO'] = p_xOi

In [22]:
candidates

,ra,dec,half_light,VLT_FORS2_g,P_O_raw,P_O,p_xO
3,326.101738,-40.899771,0.458869,25.295416,73.150261,0.031633,0.992440
7,326.105365,-40.900239,1.308629,21.319569,2017.091915,0.872255,0.130361
8,326.104186,-40.900180,0.814683,24.272838,157.983590,0.068317,0.318530
9,326.106237,-40.899279,0.501247,25.474828,64.277492,0.027796,0.836039


# $p(x)$

In [14]:
p_x = np.sum(candidates.P_O * candidates.p_xO)
p_x

4.1327985693607774e-06

# Posteriors $P(O|x)$

In [15]:
P_Oix = candidates.P_O * p_xOi / p_x
candidates['P_Ox'] = P_Oix

In [16]:
candidates

,ra,dec,half_light,VLT_FORS2_g,P_O_raw,P_O,p_xO,P_Ox
3,326.101738,-40.899771,0.458869,25.295416,73.150261,0.031633,0.000000e+00,0.000000e+00
7,326.105365,-40.900239,1.308629,21.319569,2017.091915,0.872255,4.564015e-06,9.632657e-01
8,326.104186,-40.900180,0.814683,24.272838,157.983590,0.068317,2.222214e-06,3.673426e-02
9,326.106237,-40.899279,0.501247,25.474828,64.277492,0.027796,3.099763e-23,2.084783e-19
